In [ ]:
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
from fake_useragent import UserAgent
import time
import csv

In [ ]:
# first step (TWSE_stock_every_day)

In [ ]:
stock_company = pd.read_csv("/Users/elaine/Desktop/side_project/stock_price/company.csv",usecols=["main_stock_symbol","sub_stock_symbol","Company_E"],encoding='utf-8')

In [ ]:
sub_stock_symbol = stock_company["sub_stock_symbol"].to_list()
sub_stock_symbol

In [ ]:
#all
date_year = ["2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022"]
date_month = ["0101","0201","0301","0401","0501","0601","0701","0801","0901","1001","1101","1201"]

stock_data_list = []
error_data_list = []

for stock_num in sub_stock_symbol:
    #print(stock_num)
    for year in date_year:
        #print(year)   
        for month in date_month:
            url = (f"https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={year}{month}&stockNo={stock_num}&response=json&_=1679296642947")    
            ua = UserAgent()
            user_agent = ua.random
            headers = {'user-agent': user_agent}
            data = requests.get(url, headers=headers).text
            if data and "\"total\":0" in data:
                error = [str(year), str(month),str(stock_num),data]
                error_data_list.append(error)         
            else:
                stock_data_list.append(data)
        time.sleep(40)
print(stock_data_list)

In [ ]:
#save stock_data_list json file
with open("/Users/elaine/Desktop/side_project/stock_price/stock_price_2020_2021.json", "w") as outfile: 
    json.dump(stock_data_list, outfile)

In [ ]:
#save error_data_list csv file
with open('/Users/elaine/Desktop/side_project/stock_price/error_data_list_2020_2021.csv', 'w', newline='') as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["year","month","stock_number","raw_data"])
    writer.writerows(error_data_list)

In [ ]:
#csv to dataframe(error data)
error = pd.read_csv("/Users/elaine/Desktop/side_project/stock_price/error_data_list_2017_2019.csv", converters = {"month":str}) 
error

In [ ]:
#second step (if there's error data run cells) (TWSE_stock_every_day_(for_error_data).ipynb)

In [ ]:
stock_company = pd.read_csv("/Users/elaine/Desktop/side_project/stock_price/error_data_list_2017_2022_errorlis_1.csv", converters = {"month":str},encoding='utf-8')
stock_company

In [ ]:
year =stock_company["year"].astype(str)
year = year.to_list()
Date = year + stock_company["month"]
Date = Date.to_list()
stock_number = stock_company["stock_number"].to_list()

In [ ]:
data = []
data.extend(list(D) for D in zip(Date,stock_number))

In [ ]:
stock_data_list = []
error_data_list = []
for i in data:
    url = (f"https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={i[0]}&stockNo={i[1]}&response=json&_=1679296642947")
    #print(url)
    ua = UserAgent()
    user_agent = ua.random
    headers = {'user-agent': user_agent}
    data = requests.get(url, headers=headers).text
    if data and "\"total\":0" in data:
        error = [i[0][:4],i[0][4:8], i[1],data]
        error_data_list.append(error)
    else:
        stock_data_list.append(data)
               
time.sleep(5)

In [ ]:
#save error_data_list csv file
with open('/Users/elaine/Desktop/side_project/stock_price/error_data_list_2017_2022_errorlis_2.csv', 'w', newline='') as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["year","month","stock_number","raw_data"])
    writer.writerows(error_data_list)

In [ ]:
#save stock_data_list json file
with open("/Users/elaine/Desktop/side_project/stock_price/stock_price_2017_2022_errorlist_2.json", "w") as outfile: #save file
    json.dump(stock_data_list, outfile)

In [ ]:
#third step (Merge all the download stock price json file) (TWSE_stock_price_concat.ipynb)

In [ ]:
#json file to dataframe (stock price)
with open("/Users/elaine/Desktop/side_project/stock_price/stock_price_2012_2016.json","r") as f:  
    
    json_data = json.loads(f.read())
    json_str = '[' + ','.join(json_data) + ']'
stock_price_2012_2016 = pd.read_json(json_str)

In [ ]:
#json file to dataframe (stock price)
with open("/Users/elaine/Desktop/side_project/stock_price/stock_price_2012_2016_errorlist_1.json","r") as f:  
    
    json_data = json.loads(f.read())
    json_str = '[' + ','.join(json_data) + ']'
stock_price_2012_2016_errorlist_1 = pd.read_json(json_str)

In [ ]:
#concat(add) all json file 
concat_data = pd.concat([stock_price_2012_2016,stock_price_2012_2016_errorlist_1],axis=0)
concat_data

In [ ]:
#check df NAN
concat_data.isnull().sum()  

In [ ]:
#explode (transform or modify each member of an array or list into a row)
concat_data = concat.explode('data')
concat_data = pd.concat([concat_data.drop(['data'], axis=1), concat_data['data'].apply(pd.Series)], axis=1) #axis = 0(直向合併) axis = 1(左右合併)
concat_data.drop(concat_data.columns[0:2],axis=1,inplace = True)
concat_data

In [ ]:
concat_data.drop(concat_data.columns[1:4],axis=1,inplace = True)
concat_data

In [ ]:
#change column name
concat_data.columns = ["Stock_Code","Date","Trade_Volume","Trade_Value","Opening_Price","Highest_Price","Lowest_Price","Closing_Price","Change","Transaction"]
concat_data

In [ ]:
Company_list=[]
for stock_num in concat_data["Stock_Code"]:
    if stock_num == None:
        continue
    if isinstance(stock_num,str):
        code = stock_num[7:12]
    Company_list.append(code)
#print(Company_list)       

concat_data["Stock_Code"] = Company_list
concat_data

In [ ]:
#reset index
concat_data = concat_data.reset_index(drop=True)

In [ ]:
#add column (year/month/day)
#split ["Date"] column with "/"
concat_data[['year', 'month', 'day']] = concat_data["Date"].str.split("/",expand=True)
concat_data

In [ ]:
#add Republic of China era to AD
concat_data["year"] = concat_data["year"].astype(int) 
concat_data["year"] = concat_data["year"] + 1911
concat_data["year"] = concat_data["year"].astype(str) 
#condata column year/month/day in to column Date
concat_data["Date"] = concat_data["year"] + "/" + concat_data["month"] + "/" + concat_data["day"]
concat_data

In [ ]:
#sort data with specific column
concat_data.sort_values(["Stock_Code","Date"],ascending=True)

In [ ]:
#save data to csv
concat_data.to_csv(f"/Users/elaine/Desktop/side_project/stock_price/final.csv",index = False,encoding='utf-8')